**Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».**
```
SELECT COUNT(p.id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
WHERE p.post_type_id = 1
AND (favorites_count >= 100 OR score > 300)
```
**Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.**
```
SELECT ROUND(AVG(count))
FROM 
(SELECT creation_date::date, COUNT(id)
FROM stackoverflow.posts
WHERE (creation_date::date BETWEEN '2008-11-01' AND '2008-11-18')
AND post_type_id = 1
GROUP BY creation_date::date) count_posts
```
**Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.**
```
SELECT count(DISTINCT u.id)
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE u.creation_date::date = b.creation_date::date
```
**Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?**
```
SELECT count(DISTINCT p.id)
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON p.user_id = u.id
JOIN stackoverflow.votes v ON v.post_id = p.id
WHERE u.display_name = 'Joel Coehoorn'
```
**Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.**
```
SELECT *
FROM 
(SELECT *,
ROW_NUMBER () over(order by id DESC) as rank
FROM stackoverflow.vote_types) as a
ORDER BY 1
```
**Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.**
```
SELECT u.id, count(vote_type_id)
FROM stackoverflow.users u
JOIN stackoverflow.votes v ON u.id =v.user_id
JOIN stackoverflow.vote_types vt ON vt.id = v.vote_type_id
WHERE vt.name = 'Close'
GROUP BY u.id
order by 2 DESC, 1 DESC
LIMIT 10
```
**Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.**
```
SELECT *
FROM
( SELECT *,
DENSE_RANK() Over(order by count DESC)
FROM 
(SELECT u.id, count(b.id)
FROM stackoverflow.users u 
JOIN stackoverflow.badges b ON b.user_id=u.id
WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY u.id) a) b
ORDER BY 2 DESC, 1
limit 10
```
**Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.**
```
SELECT title, user_id, score,
ROUND(avg(score) over (partition by user_id))
FROM stackoverflow.posts
WHERE title is not NULL 
AND score != 0
```
**Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.**
```
SELECT title
FROM stackoverflow.posts
WHERE user_id = (SELECT user_id
FROM stackoverflow.badges
GROUP BY user_id
HAVING count(id) > 1000 )
AND title IS NOT NULL
```
**Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.**
```
SELECT id, views,
case
WHEN views >= 350 THEN 1
WHEN views < 350 AND views >= 100 THEN 2
WHEN views < 100 THEN 3
end
FROM stackoverflow.users
WHERE location like '%United States%' AND views != 0
```
**Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.**
```
SELECT b.id,b.case, b.views
FROM (SELECT *, 
max(views) over(partition by a.case)
FROM 
(SELECT id, views,
case
WHEN views >= 350 THEN 1
WHEN views < 350 AND views >= 100 THEN 2
WHEN views < 100 THEN 3
end
FROM stackoverflow.users
WHERE location like '%United States%' AND views != 0) a) b
WHERE b.views = b.max
ORDER BY 3 desc, 1
```
**Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.**
```
SELECT extract (day from creation_date::date), a.count,
SUM(a.count) over(order by a.creation_date)
FROM (SELECT creation_date::date, count(id)
FROM stackoverflow.users
WHERE to_char(creation_date, 'yyyy-mm') = '2008-11'
group by creation_date::date) a
```
**Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.**
```
SELECT DISTINCT user_id,
a.min - a.creation_date
FROM 
(SELECT p.user_id,u.creation_date,
min(p.creation_date) over(partition by p.user_id order by p.creation_date)
FROM stackoverflow.posts p
join stackoverflow.users u ON p.user_id = u.id) a
```
**Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.**
```
SELECT date_trunc ('month', creation_date)::date, sum(views_count)
FROM stackoverflow.posts
WHERE extract (year from creation_date::date) = 2008
group by date_trunc ('month', creation_date)::date
order by 2 desc
```
**Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.**
```
SELECT u.display_name, count(Distinct u.id)
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE p.post_type_id = 2
AND (p.creation_date::date >= u.creation_date::date and p.creation_date::date <= u.creation_date::date + interval '1 month')
group by u.display_name
HAVING count(p.id)>100
order by 1
```
**Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.**
```
SELECT count(id), date_trunc('month', creation_date)::date
FROM stackoverflow.posts
WHERE user_id IN
(SELECT Distinct  p.user_id
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON p.user_id = u.id
WHERE EXTRACt (month from u.creation_date::date) = 9
AND EXTRACT (month from p.creation_date::date) = 12)
AND EXTRACT(year from creation_date::date) = 2008
Group by date_trunc('month', creation_date)::date
order by 2 desc
```
**Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.**
```
SELECT user_id, creation_date, views_count,
SUM(views_count) over(partition by user_id order by creation_date)
FROM stackoverflow.posts
order by 1
```
**Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.**
```
SELECT ROUND(AVG(a.count))
FROM (SELECT user_id , count(DIstinct creation_date::date)
FROM stackoverflow.posts
WHERE creation_date::date <= '2008-12-7' AND creation_date::date >= '2008-12-01'
GROUP BY user_id
Having count(DIstinct creation_date) > 0) a
```
**На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
номер месяца;
количество постов за месяц;
процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.**
```
With b as 
(SELECT *,
LAG(a.count) over(order by a.date_part),
round(((a.count * 1.0 / LAG(a.count) over(order by a.date_part) * 1.0) -1) * 100, 2) as percent
FROM (
    SELECT extract (month from creation_date:: date), count(id)
FROM stackoverflow.posts
WHERE creation_date >= '2008-09-01' AND creation_date <= '2008-12-31'
GROUP BY extract (month from creation_date:: date)
) a)

SELECT date_part, b.count as count_post_per_month, b.percent
FROM b
```
**Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.**
```
SELECT EXTRACT(WEEK FROM creation_date),
       MAX(creation_date)
FROM stackoverflow.posts
WHERE user_id=
    (WITH tt AS
       (SELECT user_id,
               COUNT(id) count_p
        FROM stackoverflow.posts
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 1) SELECT user_id
     FROM tt) AND DATE_TRUNC('DAY', creation_date)::date BETWEEN '2008-10-01' AND '2008-10-31'
GROUP BY 1 
```